In [1]:
import pathlib
import re

In [2]:
from pymongo import MongoClient

In [3]:
for_language = '''
                <!DOCTYPE html>
                    <head>
                            <meta charset="utf-8">
                            <title>Tasks</title>
                        <style>
                            html *,
                            hr {
                                font-size: 16px;
                                line-height: 2.625;
                                color: #000000;
                                font-family: Nunito, sans-serif;
                                text-align: justify;
                            }

                            hr {
                                display: block;
                                margin-top: 0em;
                                margin-bottom: 1.5em;
                                margin-left: auto;
                                margin-right: auto;
                                border-style: inset;
                                border-width: 1px;
                                width: 100%;
                            }

                            ::selection {
                                background-color: #000;
                                color: #fff;
                            }
                            table {
                                border-collapse: collapse; /* This removes spacing between table cells */
                                width: 100%;
                                border: 2px solid black; /* Border around the table */
                            }

                            th, td {
                                border: 1px solid black; /* Border around header cells and data cells */
                                padding: 8px; /* Add some padding for cell content */
                                text-align: left; /* Align text within cells */
                            }

                            th {
                                background-color: #f2f2f2; /* Background color for header cells */
                            }
                        </style>
                    </head>
    '''

In [4]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end='\n\n')

In [5]:
def find_task(task, num):
    file_name = 'task_'  + str(task) + '.html'
    path = pathlib.Path.cwd() /'files' / 'tasks' / file_name
    with open(path, 'r') as file:
        src = ' '.join(file.readlines())
        idx_start = src.find(f'Задание {str(num)}')
        idx_end = src.find(f'Задание {str(num + 1)}')

    return src[idx_start:idx_end]

In [6]:
def find_answer(task, num):
    file_name = 'task_'  + str(task) + '.html'
    path = pathlib.Path.cwd() /'files' / 'answers' / file_name
    with open(path, 'r') as file:
        src = ' '.join(file.readlines())
        idx_start = src.find(f'Задание {str(num)}')
        idx_end = src.find(f'Задание {str(num + 1)}')
        answer = re.findall("<br>Answer: <br>([^']*?)<hr>", src[idx_start:idx_end])[0].strip()
        if task == 25:
            answer = answer.replace('<br/>', ' | ').split('|')
            answer = '|'.join([elem.strip() for elem in answer])
        if task == 19:
            answer = answer.replace('1)', '').replace('2)', '').replace('3)', '').replace('<br/>', '|').split('|')
            answer = '|'.join([elem.strip() for elem in answer])
    return answer

In [7]:
def add_to(coll_num, task, total):
    for num in range(1, total):
        try_find = coll_num.find_one({'num': num})
        if try_find:
            print(f'Nummer {num} of task {task} is already added')
        else:
            coll_num.insert_one({'num': num, 'ans': find_answer(task, num)})

In [8]:
def add_task(coll_num, task, total):
    for num in range(1, total):
        try_find = coll_num.find_one({'num': num})
        if try_find:
            print(f'Nummer {num} of task {task} is already added')
        else:
            coll_num.insert_one({'num': num, 'task': for_language + find_task(task, num)})

In [10]:
client = MongoClient(host='localhost', port=27017, tz_aware=True)
mongodb_uri = 'mongodb://localhost:27017/'
db_name_answers = 'inf_answers'
db_name_tasks = 'inf_tasks'
db_answers = client[db_name_answers]
db_tasks = client[db_name_tasks]

In [16]:
task = 11
total = 134
coll_num = db_answers[f'{task}']
coll_num.delete_many({})
add_to(coll_num, task, total)
coll_num = db_tasks[f'{task}']
coll_num.delete_many({})
add_task(coll_num, task, total)